In [53]:
#import data from Victor
%store -r df
df['month_name'] = df.index.month_name()

#list_var_rawbox_check = ['battery_voltage', 'current_out', 'state_of_charge_percent', 'temperature', 'power_out']
option_resampling = [{'label':'month','value':'1MS'},{'label':'day','value':'1D'}]

In [54]:
df.head()

,battery_voltage,current,current_in,current_out,panel_voltage,state_of_charge_percent,temperature,usb_current,voltage,battery_id,power_out,usb_power,power_in,net_power_battery,other_out_current,month_name
timestamp,,,,,,,,,,,,,,,,
2021-03-02 06:41:24,7.075,-2.276,2.278,0.002,7.350,100.0,28.287937,0.002,7.280,0,0.01415,0.01415,16.11685,-16.102699,0.00,March
2021-03-02 06:51:24,6.720,0.000,0.002,0.002,0.005,102.0,27.152666,0.002,6.735,0,0.01344,0.01344,0.01344,0.000000,0.00,March
2021-03-02 07:01:24,6.710,0.010,0.002,0.012,0.005,102.0,26.919203,0.002,6.725,0,0.08052,0.01342,0.01342,0.067100,0.01,March
2021-03-02 07:11:24,6.705,0.010,0.002,0.012,0.005,102.0,27.003891,0.002,6.720,0,0.08046,0.01341,0.01341,0.067050,0.01,March
2021-03-02 07:21:24,6.705,0.010,0.002,0.012,0.005,102.0,27.017624,0.002,6.715,0,0.08046,0.01341,0.01341,0.067050,0.01,March


In [55]:
#!pip install dash-bootstrap-components
#!pip install jupyter-dash

In [56]:
from jupyter_dash import JupyterDash 
from dash import dcc
from dash import html
from dash.dependencies import Input, Output 
import plotly.express as px
import dash_bootstrap_components as dbc
from imp import reload

In [57]:
from plot_functions import median_profiles, plot_times_series, boxplot
reload(median_profiles)
reload(plot_times_series)
reload(boxplot)

<module 'plot_functions.boxplot' from 'c:\\Users\\PC_victor\\Documents\\6_side_projects\\battery_dev\\BatteryDevBBoxx\\notebooks\\report\\plot_functions\\boxplot.py'>

In [58]:
options_n_random_batteries = [{'label':f'{c} random batteries','value': c} for c in range(1, df.battery_id.nunique(), 5)]
#option_resampling = [{'label':'month','value':'1MS'},{'label':'day','value':'1D'}]
option_rolling = [{'label':'month','value':'30D'},{'label':'week','value':'7D'},{'label':'day','value':'1D'}]


In [59]:
#Building the app
app = JupyterDash(__name__, external_stylesheets = [dbc.themes.LUX],
                            meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}]) #if you want to make it adjust when the dashboard is visualized on mobile phones

In [60]:
#Building the layout
app.layout = dbc.Container([
    #------------------------------------------------------------------------------------------------------------------------
                    #Row #1: title row
                    dbc.Row([
                        html.H1('BBoxx Lithium-Ion Battery Field Data Challenge', className = 'text-center mb-4'),
                        html.P('Team 22: Nicole Schauser, Victor Bossard, Etienne Beauchamp, Elias Galiounas, Maria Varini', className = 'text-center m-4')
                    ]), #close row 1
    #------------------------------------------------------------------------------------------------------------------------
                    #Row 2; sub-section title and text
                    dbc.Row([
                        dbc.Col([
                      html.H2('Time series analysis', className = 'text-center mb-4 bg-primary text-white'),
                      html.P("In this section, the raw data provided can be investigated interactively both in the form of a time series and as a statistic (box plot)", className = 'text-center m-4'),
                      html.Label(['y-axis selection']),
                      dcc.Dropdown(id = 'var-dropdown',
                      value = 'voltage',
                       options = [{'label': a, 'value': a} for a in df.columns], className = 'm-1'),
                       #html.Label(['battery_id selection']),
                       #dcc.Dropdown(id = 'id-dropdown',
                        #            value = '1',
                        #            options = [{'label': b, 'value': b} for b in df['battery_id'].unique()], className = 'm-1'),
                        html.Label(['number batteries to plot']),
                        dcc.Dropdown(id = 'n_batteries-dropdown', multi = False, value = 5, options = options_n_random_batteries),
                        html.Label(['rolling aggregation']),
                        dcc.Dropdown(id = 'rolling-dropdown',
                                    value = '7D',
                                    options = option_rolling)
                    ], width = 12)
                    ]), #close row 2
     #------------------------------------------------------------------------------------------------------------------------
                      #Row 3: time series data plots
                    dbc.Row([
                        #dcc.Dropdown(id = 'field_profile-dropdown', multi = False, value = 'net_power_battery', options = options_fields),
                        #dcc.Dropdown(id = 'n_batteries_profile-dropdown', multi = False, value = 5, options = options_n_random_batteries),
                        dcc.Graph(id = 'ts-graph', figure = {})      
                    ]),
    #-------------------------------------------------------------------------------------------------------------------------
                    html.P("General statistics of the data", className = 'text-center m-4'),
    #-------------------------------------------------------------------------------------------------------------------------
                    dbc.Row([
                        #========================================================================================
                        #Column 1 (left)
                        dbc.Col([
                        dcc.Graph(id= 'profile-graph')],
                        width = {'size':5, 'offset': 1}), #close column 1
                        #===========================================================================================
                        #Column 2 (right)
                        dbc.Col([
                            dcc.Graph(id = 'stat-boxplot')
                        ], 
                        width = {'size':5, 'offset': 1})
                        ]), #close Row 3
    #------------------------------------------------------------------------------------------------------------------------
                    #Row 4: sub section title and text
                    dbc.Row([
                        dbc.Col([
                      html.H2('Clustering analysis results', className = 'text-center mb-4 bg-success text-white'),
                      html.P("Some explanation here", className = 'text-center mb-4'),
                      ], width = 12)
                    ]), #close row 4
    #-------------------------------------------------------------------------------------------------------------------------
                    #Row 5: clustering plots
                    dbc.Row([
                        #=======================================================================================
                        #Column 1
                        dbc.Col([
                            
                            '''
                             Etienne
                            
                            Please add here the "dynamic" plot with the clusters being formed.
                            '''   
                        ], width = 5), # close column #1
                        #========================================================================================
                        #Column 2
                        dbc.Col([
                                                        '''
                             Elias
                            
                            Please add here the heatmaps of your clustering.
                            '''  
                        ], width = 5) #close column 2
                    ]), #close Row 5
    #------------------------------------------------------------------------------------------------------------------------
                #Row 6: sub title and text
                      dbc.Row([
                        dbc.Col([
                      html.H2('Extracting user profiles', className = 'text-center mb-4 bg-info text-white'),
                      html.P("Some explanation here", className = 'text-center mb-4'),
                      ], width = 12)
                    ]), #close row 6
    #------------------------------------------------------------------------------------------------------------------------
                #Row 7: User profile plots
                    dbc.Row([
                        #=======================================================================================
                        #Column 1
                        dbc.Col([
                            
                            '''
                             Etienne
                            
                            Please add here the average current and voltage coming out of the clustering analysis.
                            '''   
                        ], width = 7), # close column #1
                        #========================================================================================
                        #Column 2
                        dbc.Col([
                                                        '''
                             Maria
                             
                             Score card?
                            '''  
                        ], width = 5) #close column 2
                    ]), #close Row 7
]) #close the dbc container


## Callbacks

In [61]:
@app.callback(
    Output(component_id = 'profile-graph', component_property = 'figure'),
    [
    Input(component_id = 'var-dropdown', component_property = 'value'),
    Input(component_id = 'n_batteries-dropdown', component_property = 'value'),
    ]
)
def callback_plot_median_profile(field, n_batteries):
    return median_profiles.plot(field, n_batteries, df)

In [62]:
@app.callback(
    Output(component_id = 'ts-graph', component_property = 'figure'),
    [
    Input(component_id = 'n_batteries-dropdown', component_property = 'value'),
    Input(component_id = 'var-dropdown', component_property = 'value'),
    Input(component_id = 'rolling-dropdown', component_property = 'value'),
    ]
)
def callback_plot_time_series(n_batteries, selected_var, rolling):
    return plot_times_series.plot(n_batteries, selected_var, rolling, df)

In [63]:
#callback + function for box plot (plot #2)
@app.callback(
    Output(component_id = 'stat-boxplot', component_property = 'figure'),
    [
    Input(component_id = 'var-dropdown', component_property = 'value'),
    #Input(component_id = 'resampling-dropdown', component_property = 'value'), useless => the daily plot is shitty
    ]
)
def callback_plot_boxplot(selected_var, resampling = "1MS"):
    return boxplot.plot(selected_var, resampling, df)


In [64]:
if __name__ == '__main__':
    app.run_server(debug = True, port = 1000)

Dash app running on http://127.0.0.1:1000/


C:\Users\PC_victor\Anaconda3\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



In [100]:
''' #callback + function for time series plot (plot #1)
@app.callback(
    Output(component_id = 'ts-graph', component_property = 'figure'),
    [
    Input(component_id = 'id-dropdown', component_property = 'value'),
    Input(component_id = 'var-dropdown', component_property = 'value'),
    Input(component_id = 'resampling-dropdown', component_property = 'value'),
    ]
)

def update_figure(selected_id, selected_var, resampling):
    df_plot = df[df['battery_id'] == selected_id]
    df_plot = df_plot.resample(resampling).mean()
    line_fig = px.line(df_plot.reset_index(), x = 'timestamp', y = selected_var, title = 'Time series plot')
    line_fig.update_layout(title_x=0.5, plot_bgcolor = '#d2d9d7') #https://community.plotly.com/t/how-can-i-change-the-background-color/39007/2
    #line_fig.show()
    return line_fig '''

" #callback + function for time series plot (plot #1)\n@app.callback(\n    Output(component_id = 'ts-graph', component_property = 'figure'),\n    [\n    Input(component_id = 'id-dropdown', component_property = 'value'),\n    Input(component_id = 'var-dropdown', component_property = 'value'),\n    Input(component_id = 'resampling-dropdown', component_property = 'value'),\n    ]\n)\n\ndef update_figure(selected_id, selected_var, resampling):\n    df_plot = df[df['battery_id'] == selected_id]\n    df_plot = df_plot.resample(resampling).mean()\n    line_fig = px.line(df_plot.reset_index(), x = 'timestamp', y = selected_var, title = 'Time series plot')\n    line_fig.update_layout(title_x=0.5, plot_bgcolor = '#d2d9d7') #https://community.plotly.com/t/how-can-i-change-the-background-color/39007/2\n    #line_fig.show()\n    return line_fig "

In [101]:
''' #callback + function for time series plot (plot #1)
@app.callback(
    Output(component_id = 'ts-graph', component_property = 'figure'),
    [
    Input(component_id = 'id-dropdown', component_property = 'value'),
    Input(component_id = 'var-dropdown', component_property = 'value'),
    Input(component_id = 'resampling-dropdown', component_property = 'value'),
    ]
)

def update_figure(selected_id, selected_var, resampling):
    df_plot = df[df['battery_id'] == selected_id]
    df_plot = df_plot.resample(resampling).mean()
    line_fig = px.line(df_plot.reset_index(), x = 'timestamp', y = selected_var, title = 'Time series plot')
    line_fig.update_layout(title_x=0.5, plot_bgcolor = '#d2d9d7') #https://community.plotly.com/t/how-can-i-change-the-background-color/39007/2
    #line_fig.show()
    return line_fig '''
    

" #callback + function for time series plot (plot #1)\n@app.callback(\n    Output(component_id = 'ts-graph', component_property = 'figure'),\n    [\n    Input(component_id = 'id-dropdown', component_property = 'value'),\n    Input(component_id = 'var-dropdown', component_property = 'value'),\n    Input(component_id = 'resampling-dropdown', component_property = 'value'),\n    ]\n)\n\ndef update_figure(selected_id, selected_var, resampling):\n    df_plot = df[df['battery_id'] == selected_id]\n    df_plot = df_plot.resample(resampling).mean()\n    line_fig = px.line(df_plot.reset_index(), x = 'timestamp', y = selected_var, title = 'Time series plot')\n    line_fig.update_layout(title_x=0.5, plot_bgcolor = '#d2d9d7') #https://community.plotly.com/t/how-can-i-change-the-background-color/39007/2\n    #line_fig.show()\n    return line_fig "

## Updates to the layout since I handle the code over

1. I put the title of the plotly graphs in the center. To do so, you use fig.update_layout(title_x = 0.5). Reference: https://community.plotly.com/t/title-alignment-python/30820
2. I fixed the labels to the dropdown lists. Basically, create the label as a separate object (do not include the dropdown list in it). In this way, the size of the buttons is not changed anymore. I also added a label to every dropdown list describing what they do. I added a className = 'm-1' to all of them too.
3. I modified the boxplot so that it also takes the sampling frequency into account. The daily plot looks very busy, but at least the same labels control both plots. Therefore, I also moved the dropdown list with the sampling frequency up under the other 2.
-  Victor: I actually reset that because it was indeed not informative
4. I changed the color of the headers of each section.
5. I changed the background color of the plotly plots (in the layout, same as the positioning of the text)
- Victor: the code used to plot functions is now in a folder. I import it in this notebook (cleaner)